# La reproductibilité avec les conteneurs ?

En préparant cette école, nous voulions voir comment reproduire des résultats issus d'articles. Le journal [Rescience](http://rescience.github.io/) offre la possibilité de soumettre des articles scientifiques qui se doient d'être reproductible. La plupart des dépôts sont écrits en langage Python mais sont-ils réellement reproductible. La difficulté ici est d'avoir le même environnement que les personnes qui ont écrit l'article. Mais la diversité des OS et des versions de Python, numpy, scipy, matplotlib,... peuvent nous jouer des tours !!!

Aussi nous allons essayer dans ce TP de construire un environnement dans un conteneur permettant de nous assurer que nous aurons les mêmes versions des packages utilisés lors de la rédaction de l'article. Nous nous apuierons sur cet article

https://github.com/ReScience-Archives/Shifman-2017

Je vous propose d'utiliser deux types de conteneur: Docker et Singularity. Je ne suis pas du tout un expert de ces deux technologies et il est donc fort probable que l'on peut faire mieux mais l'idée est de vous donner une première approche et de vous raconter un peu par quelles péripéties je suis passé.

## Docker

Nous allons donc commencer par Docker. L'idée que j'avais initialement est d'avoir un conteneur le plus léger possible afin d'en faciliter sa construction (notemment si vous rencontrez des problèmes de réseaux).

[DockerHub](https://hub.docker.com/) propose énormément d'images Docker. Reste à trouver la bonne !!!

Parmi les systèmes linux de base, nous trouvons ubuntu, centos, opensuse, ... Voici leur taille que l'on peut trouver sur [DockerHub](https://hub.docker.com/) dans l'onglet tag (données récoltées fin avril 2017)

- debian (sid-slim) : 23 MB
- ubuntu (latest) : 47 MB
- opensuse (latest) : 48 MB
- centos (latest) : 70 MB

Il existe d'autres images beaucoup plus légères

- busybox (latest) : 678 KB
- alpine (latest) : 2 MB

Nous allons donc utiliser une de ces images. Nous pourrions prendre busybox mais celle-ci est vraiment nue. Nous allons donc nous pencher sur alpine.

Lorsque j'ai commencé ce TP, j'ai débuté par la mise en place d'une image utilisant l'image proposée par continuum ([continuumio/miniconda](https://hub.docker.com/r/continuumio/miniconda). Elle fait de base déjà 168 MB et lorsque j'ai eu fini d'installer tout ce qu'il me fallait, je suis arrivé à une image faisant 633 MB ce qui me semblait un peu trop volumineux. J'ai donc regardé si il était possible de partir d'images plus légères et je suis tombé sur ce [blog](https://www.brianchristner.io/docker-image-base-os-size-comparison/). Les résultats datent de 2015 et n'ayant pas vérifié les tailles actuelles des différents OS, je suis partie sur l'image alpine. Celle-ci pose quelques problèmes de compatibilités avec la libc et, si je devais le refaire, je partirai probablement d'une image debian. Néamoins, l'utilisation d'alpine a été intéressante et nous resterons donc sur celle-ci pour la première partie de ce TP.

Comme dit en introduction, la plupart des articles dans Rescience sont écrits en Python. Il nous est donc nécessaire de mettre en place un environnement Python avec les packages utilisés pour reproduire les résultats. Dans l'article qui nous intéresse, les packages nécessaires sont numpy, scipy et matplotlib. Nous ajoutons à cela jupyterlab afin d'avoir un environnement agréable pour refaire les tests.

Aujourd'hui, mettre en place un environnement Python est grandement facilité par les solutions proposées par anaconda. Nous allons donc installer un miniconda dans notre conteneur et créer notre environement de travail. Le premier problème rencontré est qu'alpine n'est pas un linux standard et que la version de base ne permet pas d'installer correctement miniconda. Ceci est lié à l'utilisation par défaut de musl qui est en conflit avec des projets utilisant la glibc. Il est donc nécessaire d'installer une bonne version de glibc.

Heureusement DockerHub est notre ami et il y a donc bien l'image qu'il nous faut ([frolvlad/alpine-glibc](https://hub.docker.com/r/frolvlad/alpine-glibc/)). Vous pouvez d'ailleurs regarder le Dockerfile pour vous donner une idée de ce qui est fait pour contourner le problème de musl. Nous allons donc partir de celle-ci.

Comme vous l'avez vu, il est facile de créer un Dockerfile à partir d'une image se trouvant sur DockerHub. Il suffit de débuter le fichier par

```
FROM frolvlad/alpine-glibc
```

Nous allons commencer par rappeler quelques commandes utiles pour utiliser Docker puis par créer une image de base qui nous servira tout au long de ce TP. 

### Les commandes de base Docker

todo

### Ma première image Docker

Le module qui se charge d'installer les packages sur alpine s'appelle [apk](https://wiki.alpinelinux.org/wiki/Alpine_Linux_package_management). Nous avons besoin d'installer un certain nombre de paquets pour bien faire fonctionner notre image par la suite. En voici la liste

- bash 
- curl
- git 
- ca-certificates 
- tini@testing 
- libice 
- libsm 
- libstdc++ 

Nous allons également installer miniconda sur cette image minimal. Vous pouvez trouver la liste à jour [ici](https://repo.continuum.io/miniconda/).

Voici à quoi ressemble notre Dockerfile

In [1]:
%%file Dockerfile
FROM frolvlad/alpine-glibc

MAINTAINER Loic Gouarin "loic.gouarin@gmail.com"

# Configure environment
ENV CONDA_DIR=/opt/conda CONDA_VER=4.3.11
ENV PATH=$CONDA_DIR/bin:$PATH SHELL=/bin/bash LANG=C.UTF-8

# Install useful packages 
RUN apk --update add \
    bash \
    curl \
    git \
    ca-certificates \
    tini \
    libice \
    libsm \
    libstdc++ &&\
    rm -rf /var/cache/apk/*

# get and install miniconda
RUN curl https://repo.continuum.io/miniconda/Miniconda3-${CONDA_VER}-Linux-x86_64.sh  -o mconda.sh && \
    /bin/bash mconda.sh -f -b -p $CONDA_DIR && \
    rm mconda.sh && \
    rm -rf $CONDA_DIR/pkgs/*

Writing Dockerfile


Concernant miniconda, nous téléchargeons une version particulière spécifiée par la variable *CONDA_VER*. Les options utilisées sont pour

- f : ne renvoie pas d'erreur si le répertoire d'installation existe déjà,
- b : installe sans demander une intervention humaine,
- p : répertoire d'installation.

Un point important lorsque l'on construit des images Docker est de minimiser le plus possible la place accupée par l'installation des différentes applications. Vous noterez donc que l'on efface à chaque fois ce qui n'est plus nécessaire (cf les **rm** dans le Dockerfile).

Pour construire cette image, vous avez deux options

#### J'ai de la chance, le réseau est bon !!!

Il vous suffit allors d'exécuter la commande **build** là où se trouve le fichier *Dockerfile*

```bash
docker build -t votre_nom/alpine-base .
```

Une fois que l'image est construite, vous pouvez la tester en faisant par exemple

```bash
docker -it votre_nom/alpine-base /bin/bash
```

#### Euh, je me suis peut-être un peu emporté !!

Si le réseau n'est pas bon, il est possible de laisser Docker faire la génération de l'image pour vous en passant par GitHub. La premère chose à faire est de se créer un compte GitHub (si vous n'en avez pas déjà un, c'est par [ici](https://github.com/)).

Créez vous ensuite un dépôt git où vous mettrez par la suite votre Dockerfile.

Vous devez ensuite aller sur [DockerHub](https://hub.docker.com/) pour vous créer également un compte. Une fois connecté, allez dans l'onglet *create->create automated build* et donnez les droits à DockerHub sur votre compte GitHub et connectez le dépôt que vous venez de créer.

Poussez votre Dockerfile sur le dépôt et vous devriez voir sur DockerHub que votre image est en attente de build dans l'onglet *Build Details*.

Maintenant que nous avons construit notre image de base, nous allons créer notre environnement de travail. Il est possible de se créer des environnements différents avec la commande *conda* en spécifiant un fichier *environment.yml*. Pour plus d'informations, vous pouvez aller [ici](https://conda.io/docs/using/envs.html).

Nous allons créer l'environnement suivant

In [4]:
%%file environment.yml
name: precis
channels:
  - defaults
  - conda-forge
dependencies:
  - python=2.7
  - libgcc
  - nomkl
  - numpy
  - scipy
  - matplotlib
  - jupyterlab

Writing environment.yml


Quelques précisions sur ce fichier, il est possible de définir un certain nombre de channels permettant d'aller chercher des packages sur d'autres dépôts que le *defaults* proposé par anaconda. Ici, nous avons besoin de *conda-forge* pour avoir une version de jupyterlab. Comme vous pouvez le voir dans les *dependencies*, il est possible de spécifier des versions en utilisant *=* (cf la version de python). Nous prendrons ici les versions de numpy, scipy et matplotlib du moment (ce n'est pas très reproductible tout ça !!). Nous verrons en conclusion comment faire mieux.

Une fois que l'on a créé ce fichier, nous allons construire une nouvelle image basée sur notre image de base et créant un nouvel environnement avec *conda* appelé precis.

Il suffit donc de copier le fichier environment.yml dans le conteneur à l'aide de la commande *COPY* et d'exécuter la commande *conda* qui va bien

In [ ]:
COPY environment.yml environment.yml
RUN conda env create -f environment.yml &&\
    rm -rf $CONDA_DIR/pkgs

Enocore une fois, on libère la place des fichiers inutiles (*rm -rf $CONDA_DIR/pkgs*). Il faut donc activer l'environnement que l'on a créé en mettant les bonnes variables d'environnement dans le conteneur à l'aide de la commande *ENV*

In [ ]:
ENV PATH=$CONDA_DIR/envs/precis/bin:$PATH
ENV CONDA_ENV_PATH=$CONDA_DIR/envs/precis
ENV CONDA_DEFAULT_ENV=precis

Nous allons à présent créer un utilisateur **precis** pour que le conteneur ne soit pas exécuté en root, exposé le port *1234* pour l'utilisation de jupyterlab et définir la commande de base lorsque l'on exécute le conteneur.

In [ ]:
RUN adduser -s /bin/bash -D precis

EXPOSE 1234
# Configure container startup
ENTRYPOINT ["tini", "--"]
CMD ["jupyter", "lab", "--ip=*", "--port=1234" ,"--no-browser"]

Il ne nous reste plus qu'à passer de root à l'utilisateur precis via la commande *USER*, de cloner le dépôt de l'article qui nous intéresse via la commande *git* et de mettre une bonne configuration de matplotlib pour ne pas avoir besoin de X11.

In [ ]:
USER precis

# Clone the shifman files into the docker container
RUN git clone https://github.com/ReScience-Archives/Shifman-2017.git shifman

RUN mkdir -p /home/precis/.config/matplotlib &&\
    echo "backend      : Agg" >> /home/precis/.config/matplotlib/matplotlibrc    

Voici ce que donne le Dockerfile complet

In [6]:
%%file Dockerfile
FROM gouarin/alpine-base

MAINTAINER Loic Gouarin "loic.gouarin@gmail.com"

ENV USER=precis

COPY environment.yml environment.yml
RUN conda env create -f environment.yml &&\
    rm -rf $CONDA_DIR/pkgs

ENV PATH=$CONDA_DIR/envs/$USER/bin:$PATH
ENV CONDA_ENV_PATH=$CONDA_DIR/envs/$USER
ENV CONDA_DEFAULT_ENV=$USER

RUN adduser -s /bin/bash -D $USER

WORKDIR /home/$USER
EXPOSE 1234
# Configure container startup
ENTRYPOINT ["tini", "--"]
CMD ["jupyter", "lab", "--ip=*", "--port=1234" ,"--no-browser"]

USER $USER

RUN mkdir -p /home/$USER/.config/matplotlib &&\
    echo "backend      : Agg" >> /home/$USER/.config/matplotlib/matplotlibrc

# Clone the shifman files into the docker container
RUN git clone https://github.com/ReScience-Archives/Shifman-2017.git shifman

Overwriting Dockerfile


Une fois que vous avez construit le conteneur par une des deux méthodes proposées plus haut, il vous suffit de faire 

```bash
docker run -p 1234:1234 -it votre_nom/alpine-shifman
```

Ouvrez une page internet avec le lien proposé. Vous êtes normalement dans l'environnement de jupyterlab !! Vous pouvez ouvrir un terminal et allez dans *shifman/code* et faire

```bash
python run.py all
```

afin de générer les figures de l'article.

![Run all](images/run.png)

Malheureusement, il y a un problème d'UTF8 avec jupyterlab pour visionner les figures au format pdf. Il est donc nécessaire de passer par un notebook et de construire une iframe laissant le soin à votre navigateur de charger le pdf, comme montré ici

![iframe](images/iframe.png)


Voilà, nous avons réussi à construire un environnemt figé permettant de reproduire les résultats de l'article !!

## Singularity

Nous allons maintenant nous intéresser à Singularity. Vous n'êtes pas sans savoir qu'il y a quelques problèmes de sécurités avec Docker. Il existe aujourd'hui différents projets qui permettent d'isoler les conteneurs

- [Singularity](http://singularity.lbl.gov/)
- [Shifter](http://www.nersc.gov/research-and-development/user-defined-images/)

et ces projets sont dédiés initialement à des applications HPC tournant sur des infrastructures de type cluster.

Nous pouvons utiliser une image Docker avec Singularity, il vous est donc tout à fait possible d'utiliser l'image que vous avez créée précédemment à travers Singularity. Mais ce n'est pas ce que j'avais en tête pour tester Singularity. En effet, ce que je voulais tester était la mise en place d'un environnement permettant de reproduire des résultats issus d'un code parallèle appelant des librairies scientifiques.

L'intérêt que je vois dans singularity est d'avoir son environnement de travail prêt à l'emploi même si l'ensemble des bibiliothèques scientifiques n'est pas installé sur la machine que l'on utilise. On travaille néanmoins dans notre espace de travail sans aller dans le conteneur ce qui nous permet d'avoir l'ensemble de nos fichiers sans passer par le montage de volume comme dans Docker (voir la commande *VOLUME* dans un Dockerfile) et tout ça de manière sécurisée sans avoir besoin d'être root !!!

J'ai voulu donc tester un code que j'avais utilisé lors d'une formation en 2014 sur [PETSc](https://www.mcs.anl.gov/petsc/) qui permet de résoudre un problème de Poisson sur une grille cartésienne. Je ne vais pas rentrer dans les détails car là n'est pas l'objet de ce TP. 

Nous allons encore une fois utiliser anaconda et conda-forge. conda-forge est assez récent et offre des fonctionnalités très intéressantes. En effet, ce dépôt n'est pas uniquement dédié aux packages Python. Il permet également d'avoir accés à des librairies pré compilées pour différents OS (linux, mac os et windows). Tout le monde peut mettre ses propres librairies sur conda-forge en suivant les instructions données sur ce [lien](https://conda-forge.github.io/#contribute).

Pour tester mon code, j'ai besoin bien évidemment de PETSc mais également de CMake et de mpi. Il y a une version de PETSc sur conda-forge qui utilise mpich. Nous allons donc utiliser cette version. Il nous restera donc à installer CMake qui est également sur conda-forge. J'ai essayé de faire fonctionner tout ça avec alpine mais il y a beaucoup trop de problèmes de compatibilité avec la libc. Nous repartirons donc d'une debian. La procédure est toujours la même que précédemment et je vous mets donc le **Dockerfile** et le fichier **environment.yml**. 

In [7]:
%%file Dockerfile
FROM debian:sid-slim

MAINTAINER Loic Gouarin "loic.gouarin@gmail.com"

# Configure environment
ENV CONDA_DIR=/opt/conda CONDA_VER=4.3.11
ENV PATH=$CONDA_DIR/bin:$PATH
ENV USER=precis

# Install useful packages 
RUN apt-get update && \
    apt-get install -y curl bzip2 build-essential && \
    apt-get clean
    
# get and install miniconda
RUN curl https://repo.continuum.io/miniconda/Miniconda3-${CONDA_VER}-Linux-x86_64.sh  -o mconda.sh
    
RUN /bin/bash mconda.sh -f -b -p $CONDA_DIR && \
    rm mconda.sh && \
    rm -rf $CONDA_DIR/pkgs/*

COPY environment.yml environment.yml
RUN conda env create -f environment.yml &&\
    rm -rf $CONDA_DIR/pkgs

ENV PATH=$CONDA_DIR/envs/$USER/bin:$PATH
ENV CONDA_ENV_PATH=$CONDA_DIR/envs/$USER
ENV CONDA_DEFAULT_ENV=$USER

RUN useradd -m $USER

WORKDIR /home/$USER
USER $USER

Overwriting Dockerfile


In [8]:
%%file environment.yml
name: precis
channels:
  - conda-forge
dependencies:
  - petsc
  - cmake

Overwriting environment.yml


J'ai pu constater quelques problèmes pour mettre correctement nos variables d'environnement dans une image Singularity mais le fait de passer par une image Docker règle le problème car les variables d'environnement de notre conteneur sont bien transmises à Singularity.

Une fois que votre image Docker est prête, il vous suffit de créer une image Singularity et de construire celle-ci à l'aide de l'image Docker. Les deux commandes sont les suivantes

```bash
singularity create test_petsc.img
singularity import test_petsc.img docker://gouarin/debian-petsc
```

Nous pouvons à présent utiliser le cmake et la librairie PETSc pour compiler notre projet qui se trouve dans le répertoire github où vous avez téléchargé cette présentation dans le répertoire Poisson. Le premier problème que j'ai rencontré lors de la compilation était des problèmes de compatibilités entre la version de PETSc que j'avais utilisée à l'époque (3.5) et la version qui est sur conda-forge (3.7). Des noms de fonctions avaient changés... Pas vraiment reproductible tout ça !!

Bref, une fois les modifications réalisées tout se passe correctement. Pour tester, il vous suffit d'aller dans le répertoire Poisson, de créer un répertoire build et d'aller dedans.

Puis de taper les commandes suivantes

```bash
singularity exec /chemin/vers/image/test_petsc.img cmake -DPETSC_INCLUDES=/opt/conda/envs/precis/include/ -DPETSC_LIBRARY=/opt/conda/envs/precis/lib/libpetsc.so ..
singularity exec /chemin/vers/image/test_petsc.img make
```

Nous avons à présent un exécutable compilé avec notre environnement. Pour l'exécuter, vous pouvez par exemple taper la commande suivante

```bash
mpiexec -n 2 /chemin/vers/image/test_petsc.img ./Cversion/poissonC -assemble -poisson_ksp_monitor -poisson_pc_type gamg
```

## Conclusion

Il est donc tout à fait possible de créer des environnements à l'aide de Docker ou de Singularity permettant de reproduire nos résultats numériques. Néanmoins, plusieurs questions se posent

- Singularity est un système jeune et donc susceptible d'évoluer à l'avenir. Est-ce que les commandes utilisées seront toujours disponibles dans quelques années. On peut se poser la même question pour Docker même si ce projet est beaucoup plus mature.

- Nous avons créé des environnements à l'aide de conda. Mais nous n'avons à aucuns moments mis les versions utilisées. Du coup, si nous relançons la construction de nos images dans quelques mois, rien ne nous garantit d'avoir les mêmes versions des packages installés. Une façon d'y remédier est que l'auteur de l'article se crée un environnement via conda et fournisse ensuite son environnement à l'aide de la commande

```bash
conda env export > environment.yml
```

A ce moment là les versions sont figées.

Bref, la construction d'un environnement stable et pérenne n'est pas encore au rendez-vous mais nous pouvons espérer que ça le devienne vu l'engouement de nos jours pour toutes les solutions de type conteneur ainsi que sur les questions de reproductibilité.

In [10]:
# execute this part to modify the css style
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()